In [1]:
# archivo:    spsi.p1.e3
# asignatura: Seguridad y Protección de Sistemas Informáticos
# práctica:   Práctica 1
# autor:      José María Martín Luque

In [2]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

# Ejercicio 3

*Elabore un guion en `Sagemath` que culmine en una función `iCRT` capaz de decidir si un sistema de ecuaciones de congruencias, enteras o polinomiales, tiene solución y proporcione la solución en caso de existir.*

## Índice

* [Ecuaciones de congruencias con enteros](#Ecuaciones-de-congruencias-con-enteros)
  * [Sistemas de ecuaciones de congruencias con enteros](#Sistemas-de-ecuaciones-de-congruencias-con-enteros)
* [Ecuaciones de congruencias con polinomios](#Ecuaciones-de-congruencias-con-polinomios)


## Ecuaciones de congruencias con enteros

Vamos a comenzar implementando una función `solCong(a, b, m)` que aceptará los parámetros enteros de una ecuación de congruencia. Sea $ax\equiv b \mod{m}$ la ecuación mencionada anteriormente. Utilizando el algoritmo descrito a continuación podemos encontrar su solución.

1. Definimos $d = (a,m)$. Con la función `xgcd(a,m)` obtenemos la tupla $(d, u, v)$, donde $u, v \in \mathbb{Z}$ son unos *coeficientes de Bezout* tales que $d = au + mv$.
2. Si $d$ no divide a $b$, la congruencia no tiene solución.
3. Si $d$ divide a $b$, una solución es $x \equiv \frac{b}{d}u \mod{\frac{m}{d}}$.
4. Podemos obtener otro representante más pequeño haciendo $x' = x \mod{\frac{m}{d}}$.

In [26]:
def solCong(a, b, m):
    x = xgcd(a, m)
    
    if (b % x[0]):
        print("No hay solución para la congruencia " + str(a) + "x = " + str(b) + "  mod " + str(m) + ".")
        return [0, 0]
    else:
        m = m // x[0]
        b = b // x[0]
        return [b*x[1] % m, m]

In [12]:
solCong(12, 6, 21)

[4, 7]

In [13]:
solCong(12, 7, 21)

No hay solución para la congruencia 12x = 7  mod 21.


[0, 0]

### Sistemas de ecuaciones de congruencias con enteros

Un sistema de congruencias es de la forma 
    \begin{align}
        l_0^0 &\equiv l_1^0 \mod l_2^0 \label{eq:cong1}\;,\\ 
        &\,\,\vdots \\
        l_0^n &\equiv l_1^n \mod l_2^n\label{eq:cong3}\;.
    \end{align}

Para resolver este sistema procedemos de la siguiente forma.

1. Resolvemos la primera congruencia con `solCong`. Obtenemos una pareja $\left(t_0^0, t_1^0\right)$ de forma que $x = t_0^0 + t_1^0k_1$, donde $k_1$ es un entero. Llamamos $s$ a la solución y la definimos inicialmente como $s := \left(s_0^0, s_1^0\right) = \left(t_0^0, t_1^0\right)$. 
2. A continuación imponemos que $x$ satisfaga la segunda ecuación: 

$$l_0^1\left(t_0^0 + t_1^0k_1\right) \equiv l_1^1 \bmod{l_2^1} \implies l_0^1t_1^0k_1 \equiv \left(l_1^1 - l_0^1t_0^0\right) \bmod{l_2^1}\,.$$

3. Ahora, viendo $k_1$ como la incógnita, volvemos a llamar a `solCong` con $a = l_0^1t_1^0$, $\,b = l_1^1 - l_0^1t_0^0$ y $\,m = l_2^1$. Obtenemos de nuevo una pareja $\left(t_0^1, t_1^1\right)$, y ahora podemos escribir:

$$x = t_0^0 + t_1^0k_1 = t_0^0 + t_1^0\left(t_0^1 + t_1^1k_2\right) = t_0^0 + t_1^0t_0^1 + t_1^0t_1^1k_2.$$

4. Tendremos entonces que $\left(s_0^1, s_1^1\right) = \left(t_0^0 + t_1^0t_0^1, t_1^0t_1^1\right) = \left(s_0^0 + s_1^0t_0^1, s_1^0t_1^1\right)$. 
5. Iteramos este proceso hasta llegar a la última ecuación, obteniendo el valor de $x$ que verifica todas las ecuaciones.

Un detalle a tener en cuenta en la implementación es el valor inicial de $s$. Como sabemos que: 

\begin{align*}
    s_1 &= s_1t_1 = t_1 &\implies s_1 = 1\,,\\
    s_0 &= s_0 + s_1t_1 = s_0 + t_1 &\implies s_0 = 0\,.
\end{align*}

Si en algún momento alguna de las congruencias no tiene solución, el sistema no tiene solución. En otro caso, devolvemos la pareja $s$, cuya interpretación es la misma que lo que devuelve `solCong`. Como argumento pasamos una lista con todos los elementos $l_i^j$ ordenados de izquierda a derecha y comenzando desde la primera ecuación hasta la última.

In [6]:
def iCRT(l):
    s = [0, 1]
    
    while (len(l) > 2 and s[1]):
        t = solCong(l[0]*s[1], l[1] - l[0]*s[0], l[2])
        s = [s[0] + s[1]*t[0], t[1]*s[1]]
        l[0:3] = [] # sustituimos del 0 al 2 por la lista vacía
    if s[1]:
        return s
    else:
        return [0, 0] # el sistema no tiene solución
        

Probamos la función con el sistema de congruencias 
    \begin{align*}
        2x &\equiv 10 &\mod 12\;,\\ 
        x &\equiv 2 &\mod 21 \;.
    \end{align*}

In [7]:
iCRT([2, 10, 12, 1, 2, 21])

[23, 42]

La solución de este sistema es por tanto $x = 23 \mod 42$, que es $23$.

## Ecuaciones de congruencias con polinomios

La función que utilizamos para resolver las ecuaciones —`solCong`— está implementada sobre la función de Sage `xgcd`, que ya acepta polinomios con coeficientes en un cuerpo. Por tanto no tenemos que modificar nuestro código y podemos pasar como argumento directamente polinomios con coeficientes en un cuerpo.

Para ello comenzamos definiendo dos anillos de polinomios, $A$ sobre $\mathbb Q$ y $Z$ sobre $\mathbb F_{11}$.

In [14]:
A.<x>=QQ[]; A

Univariate Polynomial Ring in x over Rational Field

In [15]:
Z.<y>=GF(11)[]; Z

Univariate Polynomial Ring in y over Finite Field of size 11

Ahora podemos definir polinomios en estos anillos.

In [21]:
a = A(12*x + 24); a

12*x + 24

In [22]:
b = A(x^3 + 4*x + 8); b

x^3 + 4*x + 8

In [24]:
m = A(27*x + 3); m

27*x + 3

Con estos polinomios construimos la ecuación de congruencia y la resolvemos utilizando `solCong(a, b, m)`.

In [27]:
solCong(a, b, m)

[5507/16524, 27*x + 3]

También podemos resolver sistemas de ecuaciones de congruencias utilizando la función `iCRT`.

In [32]:
iCRT([Z(13), Z(5*y + 5), Z(y^3 + 8), Z(x + 1), Z(7), Z((y^2 - 27)^3)])

[y^8 + 6*y^7 + 7*y^6 + 7*y^4 + 4*y^3 + 2*y^2 + 10*y + 10,
 y^9 + 7*y^7 + 8*y^6 + 9*y^5 + y^4 + 7*y^3 + 6*y^2 + 1]